# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head(3)

In [ ]:
messages.drop(['original'], axis = 1, inplace = True)

In [ ]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head(2)

In [ ]:
# create column names using list comprehensions taken from the categories.categories Series
cols = [i.split('-')[0] for i in categories.iloc[0, 1:2].str.split(";")[0]]

In [ ]:
# testing the first element of the categories.categories gets assigned to a new column name
# the new column name is the first element of cols list cols[0] = 'related'
categories[cols[0]] = categories.categories.apply(lambda x: x.split(";")[0].split('-')[1])

In [ ]:
# create a loop to create columns with the right data for every category found in the category.category Series
import time
start = time.time()
cols = [i.split('-')[0] for i in categories.iloc[0, 1:2].str.split(";")[0]]

for col in range(0, len(cols)):
    print(f"Creating column {col + 1}: {cols[col]}")
    categories[cols[col]] = categories.categories.apply(lambda x: x.split(";")[col].split('-')[1]).astype('int64')
stop = time.time()

print(f"Calculation time: {round(stop-start, 2)} seconds")

In [ ]:
categories.head(2)

In [ ]:
# check non binalry variables
for col in cols:
    val_nums = len(categories[col].value_counts()) 
    if val_nums > 2 or val_nums < 2:
        print(f"Column {col} has {val_nums} categories")

It looks like `related` and `child_alone` variables are not binary variables

In [ ]:
categories.related.value_counts()

In [ ]:
# fix the related categories binary issue
categories.loc[(categories['related']==2)] = 1

In [ ]:
# drop the child alone category which is the same and would not add any value to our model
categories.drop(['child_alone'], axis = 1, inplace = True)

In [ ]:
categories.loc[(categories['id']==17919)]

In [ ]:
categories.loc[(categories['id']== 1)]

In [ ]:
categories[categories.duplicated()]

In [ ]:
categories.duplicated().sum()

In [ ]:
#drop dupplucates
categories.drop_duplicates(inplace = True)

In [ ]:
categories.duplicated().sum()

In [ ]:
categories.drop(['categories'], axis = 1, inplace = True)

In [ ]:
categories.head(2)

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [ ]:
# merge datasets
df = messages.merge(categories, on='id')
df.head()

In [ ]:
# drop dupplicates
df.drop_duplicates(inplace=True)

In [ ]:
#dropna
df.dropna(inplace = True)

In [ ]:
df.head(2)

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [ ]:
# create a dataframe of the 36 individual category columns
categories = df.categories.str.split(pat = ";", expand = True)
categories.head(2)

In [ ]:
%%time
# select the first row of the categories dataframe
row = categories.loc[0,:]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
row = row.str.split('-').str[0].tolist()
cat_colnames = row

# rename the columns of `categories`
categories.columns = cat_colnames

In [ ]:
# rename the columns of `categories`
categories.columns = cat_colnames
categories.head()

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [ ]:
%time        
for col in categories:
    # set each value to be the last character of the string
    categories[col] = categories[col].str.split('-').str[1]  
    # convert column from string to numeric
    categories[col] = pd.to_numeric(categories[col])
print(cat_colnames)
%time
categories.head()

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [ ]:
#drop the original categories column from `df`
df.drop(['categories'], axis = 1, inplace = True)
# no original column
categories.loc[(categories['related']==2)] = 1
df.head()

In [ ]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df.head()

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
# check number of duplicates
df.duplicated().sum()

In [ ]:
# drop duplicates
df.drop_duplicates(inplace=True)

In [ ]:
# check number of duplicates
df.duplicated().sum()

In [ ]:
# credit to https://github.com/iris-theof/Disaster_response_pipeline
def clean_data(df):
    '''
    Function to clean the dataframe in order to be compatible for the ML application.
    Split the categories column with delimit ';' 
    Convert the first row values in categories dataframe to the column headers. 
    Replace the numerical values  2 to 1.
    Drop the duplicate rows from df dataframe.
    Drop the 'child_alone' column as it always takes the same value.
    Drop the column 'original' as it will not be used for the ML model.
    Drop rows with NA.
    Remove the existing 'categories' column from the df dataframe and concat the formatted 
    categories dataframe with df dataframe.   
    
    Input: df
    Output: cleaned and formatted dataframe
    '''
    # create a dataframe of the 36 individual category columns
    categories = df["categories"].str.split(pat=";",expand=True)
    # select the first row of the categories dataframe
    row = categories.loc[0,:]
    # removed everything after -
    row = row.str.split('-').str[0].tolist()
    # use this row to extract a list of new column names for categories.                
    category_colnames = row
    # rename the columns of `categories`
    categories.columns = category_colnames
    for column in categories:
        # set each value to be the last character of the string
        categories[column] = categories[column].str.split('-').str[1]  
        # convert column from string to numeric
        categories[column] = pd.to_numeric(categories[column])
    #replace the 2 entries by 1
    categories.loc[(categories['related']==2)] = 1
    categories.drop(['child_alone'], axis=1, inplace=True)
    # drop the original categories column from `df`
    df.drop(['categories'], axis=1, inplace=True)
    # concatenate the original dataframe with the new `categories` dataframe
    df = pd.concat([df, categories], axis=1)
    # drop duplicates
    df.drop_duplicates(inplace=True)
    # drop original column as it is not needed for the ML model
    #df.drop(['original'], axis=1, inplace=True)
    # drop rows with NA
    df.dropna(inplace=True)
    
    return df

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
messages_path = 'messages.csv'
categories_path = 'categories.csv'

def load_clean(messages, categories):
    """
    Input:
    messages_path = the path of the messages.csv file,
    categories_path = the path of the categories.csv file
    
    Output:
    cleaned merged dataframe using both messages and categories data
    
    Description:
    Function to load and clean data in compatibility terms with the ML application
    1. Loads data using messages and categories paths
    3. Drops dupplicates in the messages dataframe
    3. Drops 'original' column in the messages dataframe
    4. Creates a list of column names which we find in the category dataframe, as category column
    5. Loops into all reccords of the categories data frame category column to split the category 
    column and distribute new columns in the category dataframe with the respected labels
    6. Fixes the 'related' column by replacing "2" to 1
    7. Drops 'child_alone' column which has only one value, there fore would bring no value to our model
    8. Drops categories column after having created the columns for each variable found there
    9. Merges two dataframes
    """
    import pandas as pd
    messages = pd.read_csv(messages_path)
    categories = pd.read_csv(categories_path)
    
    # drop dupplicates in messages dataframe 
    messages.drop_duplicates(inplace = True)
    
    # drop original column in the messages dataframe
    messages.drop(['original'], axis = 1, inplace = True)
    
    # create column names using list comprehensions taken from the categories.categories Series
    cols = [i.split('-')[0] for i in categories.iloc[0, 1:2].str.split(";")[0]]

    # create a loop to create columns with the right data for every category found in the category.category Series
    import time
    start = time.time()

    for col in range(0, len(cols)):
        print(f"Creating column {col + 1}: {cols[col]}")
        categories[cols[col]] = categories.categories.apply(lambda x: x.split(";")[col].split('-')[1]).astype('int64')
    stop = time.time()

    print(f"Calculation time: {round(stop-start, 2)} seconds")

    # fix the related categories binary issue
    categories.loc[(categories['related']==2)] = 1

    # drop the child alone category which is the same and would not add any value to our model
    categories.drop(['child_alone'], axis = 1, inplace = True)

    # drop categories column
    categories.drop(['categories'], axis = 1, inplace = True)

    # drop dupplicates in messages dataframe 
    categories.drop_duplicates(inplace = True)

    # merge datasets
    df = messages.merge(categories, on='id', how = 'inner')
    
    # drop nan values
    df.dropna(inplace = True)
    
    return df

In [2]:
df = load_clean(messages_path, categories_path)

Creating column 1: related
Creating column 2: request
Creating column 3: offer
Creating column 4: aid_related
Creating column 5: medical_help
Creating column 6: medical_products
Creating column 7: search_and_rescue
Creating column 8: security
Creating column 9: military
Creating column 10: child_alone
Creating column 11: water
Creating column 12: food
Creating column 13: shelter
Creating column 14: clothing
Creating column 15: money
Creating column 16: missing_people
Creating column 17: refugees
Creating column 18: death
Creating column 19: other_aid
Creating column 20: infrastructure_related
Creating column 21: transport
Creating column 22: buildings
Creating column 23: electricity
Creating column 24: tools
Creating column 25: hospitals
Creating column 26: shops
Creating column 27: aid_centers
Creating column 28: other_infrastructure
Creating column 29: weather_related
Creating column 30: floods
Creating column 31: storm
Creating column 32: fire
Creating column 33: earthquake
Creating

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [3]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///DisasterResponse.db')
df.to_sql('response_table', engine, index=False, if_exists='replace')

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.